# Tobigs 21기 4주차 정규세션 Ensemble

### 앙상블 모델 최소 2가지를 이용해 코드를 짜주세요!
- 전처리와 모델 성능결과 출력 모두 진행해주세요.
- 데이터셋: https://www.kaggle.com/datasets/sakshigoyal7/credit-card-customers
    - 다른 데이터셋을 사용해도 괜찮습니다! 대신 출처를 명시해주세요.

In [4]:
import pandas as pd

df = pd.read_csv('./data/BankChurners.csv')

# 결측치 x

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CLIENTNUM                 10127 non-null  float64
 1   Attrition_Flag            10127 non-null  int32  
 2   Customer_Age              10127 non-null  float64
 3   Gender                    10127 non-null  int32  
 4   Dependent_count           10127 non-null  float64
 5   Education_Level           10127 non-null  int32  
 6   Marital_Status            10127 non-null  int32  
 7   Income_Category           10127 non-null  int32  
 8   Card_Category             10127 non-null  int32  
 9   Months_on_book            10127 non-null  float64
 10  Total_Relationship_Count  10127 non-null  float64
 11  Months_Inactive_12_mon    10127 non-null  float64
 12  Contacts_Count_12_mon     10127 non-null  float64
 13  Credit_Limit              10127 non-null  float64
 14  Total_

In [5]:
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


In [6]:
def summary(df):
    summary_df = pd.DataFrame(df.dtypes, columns=['dtypes'])
    summary_df['count'] = df.count().values
    summary_df['unique'] = df.nunique().values 
    summary_df['missing#'] = df.isna().sum() 
    summary_df['missing%'] = df.isna().sum() / len(df)
    return summary_df

summary(df).style.background_gradient(cmap='Oranges')

,dtypes,count,unique,missing#,missing%
CLIENTNUM,int64,10127,10127,0,0.000000
Attrition_Flag,object,10127,2,0,0.000000
Customer_Age,int64,10127,45,0,0.000000
Gender,object,10127,2,0,0.000000
Dependent_count,int64,10127,6,0,0.000000
Education_Level,object,10127,7,0,0.000000
Marital_Status,object,10127,4,0,0.000000
Income_Category,object,10127,6,0,0.000000
Card_Category,object,10127,4,0,0.000000
Months_on_book,int64,10127,44,0,0.000000


## 전처리

In [7]:
df = df.iloc[:, :-2]
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
num_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

#### 라벨 인코딩 / 스탠다드 스케일러

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

def data_preprocess(df):    
    global cat_cols, num_cols
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True, keep='first')
    le = LabelEncoder()
    for col in cat_cols:
        df[col] = le.fit_transform(df[col])
    scaler = StandardScaler()
    for col in num_cols:
        df[[col]] = scaler.fit_transform(df[[col]])
    return df

In [9]:
df = data_preprocess(df)

# split

In [10]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['Attrition_Flag'])
y = df['Attrition_Flag']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

xgb = XGBClassifier(random_state=42)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
acc = accuracy_score(y_test,y_pred)
print(acc)

0.9644619940769991


In [15]:
import lightgbm as lgb

lgbm = lgb.LGBMClassifier(random_state=42)
lgbm.fit(X_train, y_train)
y_pred = lgbm.predict(X_test)
acc = accuracy_score(y_test,y_pred)
print(acc)

[LightGBM] [Info] Number of positive: 6801, number of negative: 1300
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000625 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2305
[LightGBM] [Info] Number of data points in the train set: 8101, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.839526 -> initscore=1.654705
[LightGBM] [Info] Start training from score 1.654705
0.9693978282329714
